<a href="https://colab.research.google.com/github/ayagup/stablediffusion/blob/main/Optimized_Llama_2_Inference_on_TPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
! pip install "optimum-tpu>=0.1.0" "transformers>=4.40.0" "accelerate" "datasets"

  Using cached optimum_tpu-0.2.3-py3-none-any.whl.metadata (17 kB)
  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached transformers-4.46.3-py3-none-any.whl.metadata (44 kB)
INFO: pip is looking at multiple versions of torch-xla[tpu] to determine which version is compatible with other requirements. This could take a while.
  Using cached optimum_tpu-0.2.1-py3-none-any.whl.metadata (17 kB)
  Using cached optimum_tpu-0.2.0-py3-none-any.whl.metadata (17 kB)
  Using cached transformers-4.41.1-py3-none-any.whl.metadata (43 kB)
  Using cached optimum_tpu-0.1.5-py3-none-any.whl.metadata (16 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 100.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.5/561.5 kB 18.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 62.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 105.6 MB/s  0:00:00
   

In [37]:
!curl -sS https://bootstrap.pypa.io/get-pip.py -o get-pip.py

In [38]:
!python3.11 get-pip.py

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 33.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [pip]


In [39]:
!apt-get update && apt-get install python3.11

pip 25.2 from /usr/local/lib/python3.11/dist-packages/pip (python 3.11)


In [40]:
!pip install torch==2.3.0 'torch_xla[tpu]==2.3.0' -f https://storage.googleapis.com/libtpu-releases/index.html

Looking in links: https://storage.googleapis.com/libtpu-releases/index.html
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.0/115.0 MB 73.5 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 31.2 MB/s  0:00:09
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 MB 52.0 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 56.6 MB/s  0:00:05
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 151.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 159.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 31.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB ?  0:00:34
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 84.3 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 99.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 85.7 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 83.4 MB/s  0:00:02


In [1]:
#
# A complete guide for running a Llama-2 model on a Google Cloud TPU using optimum-tpu and PyTorch/XLA.
#
# This script is designed to be executed in a properly configured Google Cloud TPU VM.
# It uses torch_xla.distributed.xla_multiprocessing to launch a separate process on each
# of the 8 TPU cores, which is the most efficient way to utilize the hardware.
#
# Prerequisites (run this in your shell on the TPU VM):
# 1. Install necessary libraries, including the TPU-specific PyTorch/XLA wheels:
#    pip install optimum-tpu -f https://storage.googleapis.com/libtpu-releases/index.html
#    pip install transformers accelerate datasets torch
#
# 2. Log in to the Hugging Face Hub if the model is gated (like Llama 2):
#    from huggingface_hub import login
#    login(token="YOUR_HUGGINGFACE_TOKEN")
#
#    Note: This token should be a secure environment variable in a real-world scenario.
#

import os
import torch
import torch_xla
import torch_xla.core.xla_model as xm
import torch_xla.debug.metrics as met
import torch_xla.distributed.xla_multiprocessing as xmp

from transformers import AutoTokenizer, AutoModelForCausalLM
from accelerate import Accelerator


/usr/local/lib/python3.12/dist-packages/torch_xla/experimental/gru.py:113: SyntaxWarning: invalid escape sequence '\_'
  * **h_n**: tensor of shape :math:`(D * \text{num\_layers}, H_{out})` or


In [ ]:
# Log in to the Hugging Face Hub if the model is gated (like Llama 2).
# The user needs to manually create a token and paste it here or use
# a secret store. Replace "YOUR_HUGGINGFACE_TOKEN" with your actual token.
from huggingface_hub import login
login(token="YOUR_HUGGINGFACE_TOKEN")

In [ ]:
# The PJRT_DEVICE environment variable must be set to 'TPU' for PyTorch/XLA to
# use the TPU device. This is often set automatically in a TPU VM environment.
os.environ['PJRT_DEVICE'] = 'TPU'


def run_tpu_inference(index, args):
    """
    This function runs on each of the TPU cores. The 'index' corresponds
    to the ID of the core (0-7 for an 8-core TPU).

    Args:
        index (int): The index of the current TPU core process.
        args (tuple): A tuple containing any additional arguments passed to the function.
    """
    # Get the TPU device for the current process
    device = xm.xla_device()
    print(f"Process {index}: Initializing on device {device}")

    # Model ID for Llama-2. We'll use the 7B model as a common example.
    # The principles apply directly to the 3B model as well.
    model_id = "meta-llama/Llama-2-7b-hf"

    try:
        # Load the tokenizer
        if xm.is_master_ordinal():
            print("Loading tokenizer...")
        tokenizer = AutoTokenizer.from_pretrained(model_id)

        # Load the model with `torch.bfloat16` for optimal performance on TPUs.
        # `device_map="auto"` uses accelerate to automatically handle sharding and
        # placement of the model across all available devices (TPU cores).
        if xm.is_master_ordinal():
            print("Loading model...")
        model = AutoModelForCausalLM.from_pretrained(
            model_id,
            torch_dtype=torch.bfloat16,
            device_map="auto"
        )

        # A simple test prompt.
        prompt = "Explain the importance of TPU for large language models."
        inputs = tokenizer(prompt, return_tensors="pt").to(device)

        # Ensure all processes have loaded the model before proceeding.
        # This is crucial for distributed training/inference.
        xm.mark_step()

        # Only the master process needs to print the generation message.
        if xm.is_master_ordinal():
            print("Generating text...")

        # Generate a response.
        # The first run will trigger JIT compilation by PyTorch/XLA, which can take
        # some time but optimizes subsequent runs.
        outputs = model.generate(
            **inputs,
            max_new_tokens=100,
            do_sample=True,
            top_k=50,
            top_p=0.95,
            num_return_sequences=1
        )

        # Only the master process decodes and prints the result.
        if xm.is_master_ordinal():
            decoded_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
            print("\n" + "="*50)
            print("Generated Text:")
            print(decoded_text)
            print("="*50 + "\n")

    except Exception as e:
        print(f"An error occurred on process {index}: {e}")
        # Print TPU metrics for debugging in case of an error on any process.
        print(f"TPU Metrics (Process {index}):")
        print(met.metrics_report())


In [ ]:
if __name__ == '__main__':
    # Use xla_multiprocessing to spawn processes, one for each TPU core.
    # The number of cores is automatically detected by the system.
    num_tpu_cores = xm.xrt_world_size()
    print(f"Detected {num_tpu_cores} TPU cores. Spawning a process for each.")

    # The xmp.spawn call sets up the distributed environment and launches
    # the 'run_tpu_inference' function on each core.
    xmp.spawn(run_tpu_inference, args=({}), nprocs=num_tpu_cores, start_method='fork')